In [5]:
## company name (main)
import re
from pypdf import PdfReader 
pdf_path = r"D:\Python\ajit\project_credit_rating\try\pdf\Adani Transmission Limited.pdf"
# reader = PdfReader(pdf_path)
# page = reader.pages[0] 
# text = page.extract_text() 
text = extract_text_from_pdf(pdf_path)
pattern = re.compile(r'\b[A-Za-z0-9\s]+(?:Limited|Ltd|Corporation|Corp|Incorporated|Inc|LLC|LLP)\b', re.IGNORECASE)
companies = pattern.findall(text)

def company_name(companies):
    com_name = []
    if 'CARE' in companies[0] or 'ICRA' in companies[0]:
        companies.pop(0)
    new_first_company = companies[0]
    new_first_company = re.sub(r'Press Release\s+', '', new_first_company, flags=re.IGNORECASE)
    new_first_company = re.sub(r'^\d+\s+', '', new_first_company)
    pattern = re.compile(r'\b[A-Za-z\s]+(?:Limited|Ltd|Corporation|Corp|Incorporated|Inc|LLC|LLP)\b', re.IGNORECASE)
    matches = pattern.findall(new_first_company)
    expected_company_name = matches[0] if matches else ""
    expected_company_name = expected_company_name.replace('\n', ' ')
    com_name.append(expected_company_name)
    return com_name

'  Adani Transmission  Limited'

In [4]:
## code for Rating Agency name

from PyPDF2 import PdfReader
def extract_text_from_pdf(pdf_file_path):
    reader = PdfReader(pdf_file_path)
    page = reader.pages[0]
    text = page.extract_text()
    return text

def find_agency_names(pdf_file_path):
    text = extract_text_from_pdf(pdf_file_path)
    agencies = []
    if 'CARE' in text:
        agencies.append('CARE')
    if 'ICRA' in text:
        agencies.append('ICRA')
    if 'Brickwork rating' in text or 'brickworkrating' in text:
        agencies.append('Brickworks')
    return agencies

pdf_file_path = r"D:\Python\ajit\project_credit_rating\try\pdf\Adani Transmission Limited.pdf"
agencies = find_agency_names(pdf_file_path)
print("Agencies:", agencies)


Agencies: ['ICRA']


In [ ]:
## code for outlook - stable or positive or negative
def outlook(data):
    ratings = []
    for row in data[1:]:
        rating = row[2]
        if 'Stable' in rating:
            ratings.append('Stable')
        elif 'Positive' in rating or 'Negative' in rating:
            ratings.append(rating.split(';')[1].strip())
        else:
            ratings.append('-')

    return ratings

In [6]:
import numpy as np
def remove_commas(data):
    for row in data:
        for i in range(len(row)):
            if isinstance(row[i], str) and row[i].replace(',', '').replace('.', '').isdigit():
                row[i] = row[i].replace(',', '')
    return data


In [ ]:
## all amount extract (main)
def extract_all_amount(data):
    short_term_amounts = []
    long_term_amounts = []
    unallocated_amounts = []
    for row in data[1:]:
        facility1 = row[0]
        facility = facility1.lower()
        amount = float(row[1])
        
        if 'short term' in facility or 'long term / short term' in facility or 'short-term' in facility :
            short_term_amounts.append(amount)
            long_term_amounts.append(0.00)
            unallocated_amounts.append(0.00)
        elif 'long term' in facility or 'long-term' in facility:
            long_term_amounts.append(amount)
            short_term_amounts.append(0.00)
            unallocated_amounts.append(0.00)
        else:
            unallocated_amounts.append(amount)
            short_term_amounts.append(0.00)
            long_term_amounts.append(0.00)
    return short_term_amounts, long_term_amounts, unallocated_amounts

extract_all_amount(data)


In [2]:
## Rating (main)

import re
import numpy as np

def extract_ratings_from_data(data):
    rating_pattern = re.compile(r'\b(?:[A-Z]{1,3}[+-]?/[A-Z0-9+-]+(?:/[A-Z0-9+-]+)?(?:;Negative)?)\b|[A-Z0-9+-]+(?:;Negative)?')
    ratings = []
    for row in data:
        row_ratings = []
        for cell in row:
            if isinstance(cell, str): 
                matches = rating_pattern.findall(cell)
                valid_matches = [match for match in matches if len(match) > 1 and not match.isdigit() and match not in ['CARE', 'ICRA', 'Commercial paper (CP)','CP','CE']]
                row_ratings.extend(valid_matches)
        if row_ratings:
            ratings.append('/'.join(row_ratings))
    
    return ratings
ratings = extract_ratings_from_data(data)
print("Extracted Ratings:", ratings)


Extracted Ratings: ['BBB-/A3/A-;Negative', 'A3', 'AA+/A1+']


In [3]:
## Analyst contact details (main)

import re
from PyPDF2 import PdfReader

# pdf_path = r"D:\Python\ajit\project_credit_rating\try\pdf\Adani Agri Fresh Limited 1.pdf"
pdf_path = r"D:\Python\ajit\project_credit_rating\try\pdf\Adani Transmission Limited.pdf"
def extract_text_from_last_pages(pdf_file_path, num_pages=3):
    reader = PdfReader(pdf_file_path)
    text = ""
    total_pages = len(reader.pages)
    for page_num in range(total_pages - num_pages, total_pages):
        text += reader.pages[page_num].extract_text()
    return text

def extract_analyst_contacts(pdf_file_path):
    text = extract_text_from_last_pages(pdf_file_path)
    
    if "ANALYST CONTACT" in text:
        name_pattern = re.compile(r'(?<=\n)[A-Za-z\s]+(?=\n\+\d{2}\s\d{2,3}\s\d{7,10})')
        email_pattern = re.compile(r'[\w\.-]+@[\w\.-]+')
        phone_pattern = re.compile(r'\+\d{2}\s\d{2,3}\s\d{7,10}')
        names = name_pattern.findall(text)
        emails = email_pattern.findall(text)
        phones = phone_pattern.findall(text)
        names_list = []
        emails_list = []
        contact_numbers_list = []
        
        for name, phone in zip(names, phones):
            email_match = email_pattern.search(text, text.find(phone))
            email = email_match.group(0) if email_match else None
            names_list.append(name.strip())
            emails_list.append(email)
            contact_numbers_list.append(phone)

        return names_list, emails_list, contact_numbers_list
    else:
        return [], [], []


names, emails, contact_numbers = extract_analyst_contacts(pdf_path)
print("Names:", names)
print("Emails:", emails)
print("Contact Numbers:", contact_numbers)


Names: ['Sabyasachi Majumdar']
Emails: ['sabyasachi@icraindia.com']
Contact Numbers: ['+91 124 4545304']


In [6]:
# ## Type of rating (main)
# def type_of_rating(data):
#     facilities = []
#     for row in data[1:]:
#         facility = row[0]
#         if 'Short Term' in facility or 'Long Term / Short Term' in facility:
#             facilities.append('ST')
#         elif 'Long Term' in facility:
#             facilities.append('LT')
#         elif 'Unallocated Term' in facility:
#             facilities.append('UT')
#         else:
#             facilities.append('ST')
#     return facilities

# # type_of_rating(data)

def type_of_rating(data):
    facilities = []
    for row in data[1:]:
        facility1 = row[0]
        facility = facility1.lower()
        print(facility)
        if 'short term' in facility or 'long term / short term' in facility:
            facilities.append('ST')
        elif 'long term' in facility or 'long-term'in facility or 'long - term'in facility:
            facilities.append('LT')
        elif 'unallocated term' in facility:
            facilities.append('UT')
        else:
            facilities.append('ST')
    return facilities

['ST', 'LT', 'ST', 'ST', 'ST']

In [9]:
# code for type of facilities (main)

import re
import numpy as np
data = [
    ['Previous Rated Amount\nCurrent Rated Amount\nInstrument*\nRating Action\n(Rs. crore)\n(Rs. crore)', np.nan, np.nan, np.nan],
    ['Commercial paper (CP)', '1,000.00', '1,000.00', '[ICRA]A1+; reaffirmed'],
    ['Long Term Bank Facilities@', '100.00', 'CARE A1+', 'Reaffirmed'],
     ['Fund-based - Term Loan', '200.00', 'CARE A1+', 'Reaffirmed'],
     ['Long Term / Short Term Bank Facilities', '145.00', 'CARE BBB-; Stable / CARE A3 A-;Negative', 'Reaffirmed'],
    ['Total', '1,000.00', '1,000.00', np.nan]
]

def extract_type_of_facilities(data):
    facilities = []
    pattern = re.compile(r'Commercial paper \(CP\)|^[A-Za-z\s\-\/\(\)]+(?:Bank Facilities|Term Loan|Commercial paper|Amount)$')
    for row in data:
        if isinstance(row[0], str) and pattern.match(row[0]):
            facilities.append(row[0])
    
    return facilities

commercial_paper_facilities = extract_type_of_facilities(data)

print("Facilities:", commercial_paper_facilities)


Facilities: ['Commercial paper (CP)', 'Fund-based - Term Loan', 'Long Term / Short Term Bank Facilities']
